In [1]:
%load_ext autoreload
%autoreload 2
import sys
# sys.path.append(r'E:/0code')
sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code/mylearn')
sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code')
# sys.path.append('/Users/yanbin/Documents/Projects/AI-Middle-Project/')
# sys.path.append('/Users/yanbin/Documents/Projects/mylearn')

# sys.path.append('/home/wyf/0code/AI-Middle-Project/')
# sys.path.append('/home/wyf/0code/mylearn')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from logger import get_logger
from neural_network import MLPClassifier
from pyml.model_selection import KFold
from pyml.model_selection import ShuffleSplit
from pyml.metrics.classification import precision_score


mylogger = get_logger(__name__)
mylogger.debug('hello world')

[DEBUG] <ipython-input-1-8dd56ce29e25> <ipython-input-1-8dd56ce29e25> - <module> line 23 - 
hello world
 - (2018-10-21 02:35:18)

[DEBUG] __init__ __init__.py - pylab_setup line 90 - 
backend module://ipykernel.pylab.backend_inline version unknown
 - (2018-10-21 02:35:18)



# 读取数据

In [12]:
train_X_vec2doc_2 = np.load('../data/wyf-train_X-doc2vec-500.npy')
test_X_vec2doc_2 = np.load('../data/wyf-test_X-doc2vec-500.npy')

In [13]:
train_X_vec2doc = np.load('../data/clean/trainX_vec2doc-24000-500.npy')
test_X_vec2doc = np.load('../data/clean/testX_vec2doc-6000-500.npy')

In [2]:
train_X_tfidf = np.load('../data/clean/trainX_tfidf-24000-62761.npy')
test_X_tfidf = np.load('../data/clean/testX_tfidf-6000-62761.npy')

In [15]:
train_X_onehot = np.load('../data/clean/trainX_onehot-24000-62761.npy')
test_X_onehot = np.load('../data/clean/testX_onthot-6000-62761.npy')

In [4]:
train_ori_Y = np.load('../data/clean/trainY_24000.npy')

## 使用tfidf进行交叉验证

In [5]:
train_X = train_X_tfidf.T
test_X = test_X_tfidf.T
train_Y  = train_ori_Y.reshape((-1,1))
train_X.shape,train_Y.shape,test_X.shape

((62761, 24000), (24000, 1), (62761, 6000))

In [6]:
train_X = pd.DataFrame(train_X)
test_X = pd.DataFrame(test_X)
train_Y = pd.DataFrame(train_Y)
train_X.shape,train_Y.shape,test_X.shape

((62761, 24000), (24000, 1), (62761, 6000))

In [8]:
learning_rates = [1,0.5]
# learning_rates = [0.5, 0.3, 0.1, 0.05, 0.02, 0.01]

n_splits = 1
ms = ShuffleSplit(n_splits=n_splits,test_size=0.2)
models = []
for train_indices, valid_indices in ms.split(train_X_tfidf):
    for learning_rate in learning_rates:
        train_train_X = train_X.iloc[:,train_indices]
        train_train_Y = train_Y.iloc[train_indices,:]
        train_valid_X = train_X.iloc[:,valid_indices]
        train_valid_Y = train_Y.iloc[valid_indices,:]
        mlpc = MLPClassifier(verbose=False, 
            # 忽略输入和输出层。中间层的结点数。例如 本例子为 input -> 8 -> 4 -> 2（2分类）
            hidden_layer_sizes=(4,), 
            activation='relu', # 'relu', 'sigmoid'. 更多函数，等待支持
            max_iter=5000, # 迭代次数
            learning_rate_init=learning_rate, 
            warm_start=True,  # True就好了
            mini_batch='auto', # 保持auto
            step_size=10, # 多少个iteration后，才进行“输出loss”，输出“准确度”，保存模型文件，这三个操作?
            load_from_file=False, # 读文件恢复模型吗
            dump_file=False, # 要不要把模型写入文件？
            validation_set=(train_valid_X, train_valid_Y) # 要不要给一个validation set？（仅用来输出这个集合的准确度)
        )
        mlpc.fit(train_train_X, train_train_Y)
        r = mlpc.score(train_valid_X, train_valid_Y)
        models.append(mlpc)
        print(r)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[0] loss 0.6932099822182799
 - (2018-10-21 02:38:58)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5004166666666666, training score 0.5036458333333333
 - (2018-10-21 02:39:04)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[10] loss 0.692506830664927
 - (2018-10-21 02:39:32)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.49833333333333335, training score 0.5197916666666667
 - (2018-10-21 02:39:37)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[20] loss 0.6928512054722534
 - (2018-10-21 02:40:04)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.50125, training score 0.5145833333333333
 - (2018-10-21 02:40:09)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[30] loss 0.693066742831

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[280] loss 0.6896237964014081
 - (2018-10-21 02:53:57)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5529166666666666, training score 0.5625
 - (2018-10-21 02:54:02)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[290] loss 0.6912204988614941
 - (2018-10-21 02:54:29)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5310416666666666, training score 0.55
 - (2018-10-21 02:54:33)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[300] loss 0.6915516877517199
 - (2018-10-21 02:55:00)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5570833333333334, training score 0.5229166666666667
 - (2018-10-21 02:55:05)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[310] loss 0.6887751375878223
 - (2

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[560] loss 0.682860353837243
 - (2018-10-21 03:08:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5777083333333334, training score 0.5697916666666667
 - (2018-10-21 03:08:54)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[570] loss 0.6804588200776654
 - (2018-10-21 03:09:21)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5795833333333333, training score 0.5890625
 - (2018-10-21 03:09:26)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[580] loss 0.6805171200195379
 - (2018-10-21 03:09:53)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5829166666666666, training score 0.5885416666666666
 - (2018-10-21 03:09:58)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[590] loss 0.677982

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[840] loss 0.6583605204822083
 - (2018-10-21 03:23:39)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.61875, training score 0.6276041666666666
 - (2018-10-21 03:23:44)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[850] loss 0.6513107159267276
 - (2018-10-21 03:24:11)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6191666666666666, training score 0.6427083333333333
 - (2018-10-21 03:24:16)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[860] loss 0.6527231800044474
 - (2018-10-21 03:24:43)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6264583333333333, training score 0.6630208333333333
 - (2018-10-21 03:24:48)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[870] loss 0.6549515

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5825, training score 0.596875
 - (2018-10-21 03:37:59)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[1120] loss 0.6564410660689897
 - (2018-10-21 03:38:26)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.569375, training score 0.5796875
 - (2018-10-21 03:38:30)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[1130] loss 0.6299330896199461
 - (2018-10-21 03:38:57)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6233333333333333, training score 0.6463541666666667
 - (2018-10-21 03:39:02)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[1140] loss 0.6553863554636892
 - (2018-10-21 03:39:29)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5645833333333333, training s

...

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6260416666666667, training score 0.6578125
 - (2018-10-21 10:26:29)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[3830] loss 0.5495198172410196
 - (2018-10-21 10:26:57)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6354166666666666, training score 0.6708333333333333
 - (2018-10-21 10:27:02)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[3840] loss 0.6114923981937125
 - (2018-10-21 10:27:30)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.613125, training score 0.6140625
 - (2018-10-21 10:27:35)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[3850] loss 0.6006714475001128
 - (2018-10-21 10:28:02)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.60375, training

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4100] loss 0.6325085505828817
 - (2018-10-21 10:41:20)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.605625, training score 0.63125
 - (2018-10-21 10:41:25)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4110] loss 0.573022261362269
 - (2018-10-21 10:41:52)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.64125, training score 0.6625
 - (2018-10-21 10:41:56)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4120] loss 0.5749454868776885
 - (2018-10-21 10:42:23)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6425, training score 0.6567708333333333
 - (2018-10-21 10:42:28)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4130] loss 0.555758947423353
 - (2018-10-21 10:42:55)

[INFO] 

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.61375, training score 0.6328125
 - (2018-10-21 10:55:39)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4380] loss 0.6648790656945398
 - (2018-10-21 10:56:06)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5741666666666667, training score 0.5921875
 - (2018-10-21 10:56:10)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4390] loss 0.6166401127742299
 - (2018-10-21 10:56:37)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6133333333333333, training score 0.6203125
 - (2018-10-21 10:56:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4400] loss 0.5905187325100177
 - (2018-10-21 10:57:09)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.623125, training score 0.

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.49916666666666665, training score 0.5010416666666667
 - (2018-10-21 11:10:28)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4660] loss 1.3043425131912447
 - (2018-10-21 11:10:55)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.49895833333333334, training score 0.48854166666666665
 - (2018-10-21 11:11:00)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4670] loss 1.5593180813681011
 - (2018-10-21 11:11:27)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.49875, training score 0.5119791666666667
 - (2018-10-21 11:11:32)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4680] loss 1.7917254024123936
 - (2018-10-21 11:11:58)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation sco

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4930] loss 5.650999623042045
 - (2018-10-21 11:24:44)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5014583333333333, training score 0.509375
 - (2018-10-21 11:24:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4940] loss 5.477030728066722
 - (2018-10-21 11:25:15)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5014583333333333, training score 0.5244791666666667
 - (2018-10-21 11:25:19)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4950] loss 5.9089535017985595
 - (2018-10-21 11:25:45)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5014583333333333, training score 0.4869791666666667
 - (2018-10-21 11:25:50)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[4960] loss 5.9149

In [ ]:
for i,model in enumerate(models):
    m = model.information
    num_iterations = list(m.keys())
    costs = [m[i][0]  for i in num_iterations]
    valid_scores = [m[i][1]  for i in num_iterations]
    train_scores = [m[i][2]  for i in num_iterations]
    plt.plot(num_iterations, valid_scores, '-', label='valid '+ str(learning_rates[i]))
#     plt.plot(num_iterations, train_scores, ':', label='train '+ str(learning_rates[i]))
    plt.legend()